In [1]:
#  Updated Example with Pruning Count
import numpy as np
from tslearn.datasets import UCR_UEA_datasets
from tslearn.metrics import dtw, lb_keogh
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Load dataset
ucr = UCR_UEA_datasets()
X_train, y_train, X_test, y_test = ucr.load_dataset("GunPoint")
X_train = X_train.squeeze()
X_test = X_test.squeeze()

# 1-NN with LB_Keogh pruning and stats
class LBKeogh1NN:
    def __init__(self, lb_radius=5):
        self.lb_radius = lb_radius
        self.total_dtw_calls = 0
        self.total_pruned = 0

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_test):
        preds = []
        for x in tqdm(X_test, desc="Classifying"):
            min_dtw = float('inf')
            best_label = None
            for xt, label in zip(self.X_train, self.y_train):
                # Prune using LB_Keogh
                lb = lb_keogh(x, xt, radius=self.lb_radius)
                if lb >= min_dtw:
                    self.total_pruned += 1
                    continue

                dist = dtw(x, xt)
                self.total_dtw_calls += 1
                if dist < min_dtw:
                    min_dtw = dist
                    best_label = label
            preds.append(best_label)
        return np.array(preds)

# Run
model = LBKeogh1NN(lb_radius=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Results
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print(f"DTW calls made: {model.total_dtw_calls}")
print(f"Candidates pruned by LB_Keogh: {model.total_pruned}")
total_candidates = len(X_test) * len(X_train)
print(f"Pruning ratio: {model.total_pruned / total_candidates:.2%}")


/Users/cyuab/miniconda3/envs/ksfdtw/lib/python3.12/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
Classifying: 100%|██████████| 150/150 [00:01<00:00, 99.97it/s] 

Accuracy: 0.9400
DTW calls made: 605
Candidates pruned by LB_Keogh: 6895
Pruning ratio: 91.93%
